# 1. Train autoencoder

## Set environment
For building the neural network, Keras is use as an interface to TensorFlow. The localtion of the data file is set in the file variable.

In [3]:
import numpy as np
import keras
from keras import layers
from keras.models import save_model
from keras.callbacks import TensorBoard
# Set location of data file
file="data/data.npy"

2022-06-09 12:43:55.614896: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-09 12:43:55.614939: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Build train, validate and test datasets
After randomizing the order of the data, the dataset is split in 3 parts:
- a train dataset for training the NN with 90% of the data
- a validation dataset, used to validate the model during training, with 5% of the data
- a test dataset, which is not used during training, to evaluate the model, with 5% of the data

In [4]:
# N is the size of the test and validation dataset
N = 2000
data = np.load(file)
i = np.argsort(np.random.random(data.shape[0]))
data = data[i]
test = data[:N//2]
validate = data[N//2:N]
train = data[N:]

## Architecture of the neural network
Convolute...

In [5]:
# Encoder (32,64,2) - > (4,8,1)
input = keras.Input(shape=(32, 64, 2))
x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(input)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
encoded = layers.Conv2D(1, (3, 3), activation='relu', padding='same')(x)

# Decoder (4,8,1) -> (32,64,2)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(32, (3, 3), activation='relu',padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
decoded = layers.Conv2D(2, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = keras.Model(input, decoded)
autoencoder.compile(optimizer='adam', loss='MSE')
print(autoencoder.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 64, 2)]       0         
                                                                 
 conv2d (Conv2D)             (None, 32, 64, 32)        608       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 32, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 32, 16)        4624      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 8, 16, 16)        0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 8, 16, 8)          1160  

2022-06-09 12:44:03.902002: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-06-09 12:44:03.902029: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-06-09 12:44:03.902049: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ai): /proc/driver/nvidia/version does not exist
2022-06-09 12:44:03.902229: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Start tensorboard
Train...

In [13]:
%%bash
tensorboard --logdir=/tmp/autoencoder 2> /dev/null &
firefox 127.0.0.1:6006 &

Process is interrupted.


## Train the model
train...

In [7]:
autoencoder.fit(train, train,
                epochs=50,
                batch_size=256,
                shuffle=True,
                validation_data=(validate, validate),
                callbacks=[TensorBoard(log_dir='/tmp/autoencoder')])

autoencoder.save("data/model2.h5")
#save_model(autoencoder,"model2")
#autoencoder.save_weights("data/weights2.h5")

2022-06-09 13:02:39.217455: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 631472128 exceeds 10% of free system memory.
2022-06-09 13:02:40.416143: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 631472128 exceeds 10% of free system memory.


Epoch 1/50
151/151 [==============================] - 87s 577ms/step - loss: 0.0089 - val_loss: 0.0090
Epoch 2/50
151/151 [==============================] - 85s 564ms/step - loss: 0.0089 - val_loss: 0.0090
Epoch 3/50
151/151 [==============================] - 85s 559ms/step - loss: 0.0089 - val_loss: 0.0090
Epoch 4/50
151/151 [==============================] - 96s 638ms/step - loss: 0.0089 - val_loss: 0.0090
Epoch 5/50
151/151 [==============================] - 87s 574ms/step - loss: 0.0089 - val_loss: 0.0090
Epoch 6/50
151/151 [==============================] - 86s 573ms/step - loss: 0.0089 - val_loss: 0.0090
Epoch 7/50
151/151 [==============================] - 85s 563ms/step - loss: 0.0089 - val_loss: 0.0090
Epoch 8/50
151/151 [==============================] - 88s 585ms/step - loss: 0.0089 - val_loss: 0.0090
Epoch 9/50
151/151 [==============================] - 88s 584ms/step - loss: 0.0089 - val_loss: 0.0090
Epoch 10/50
151/151 [==============================] - 85s 565ms/step - l

KeyboardInterrupt: 

## Evaluate the model

In [ ]:
#score = autoencoder.evaluate(x_test, x_test, verbose=0)
#print('Test loss:', score)